# Day 17

In [1]:
with open('../inputs/adventofcode.com_2022_day_17_input.txt', 'r') as f:
    data = f.read().splitlines()
data = data[0]
print(f'The file contains {len(data)} movements.')

The file contains 10091 movements.


## Puzzle 1

In [2]:
import numpy as np

tunnel = np.zeros((1, 7), dtype=int)

num_rocks = 2022

move = 0

rocks = {}

rocks[0] = np.ones((1, 4), dtype=int)
rocks[1] = np.array([[0,1,0], [1,1,1], [0,1,0]], dtype=int)
rocks[2] = np.array([[0,0,1], [0,0,1], [1,1,1]], dtype=int)
rocks[3] = np.ones((4, 1), dtype=int)
rocks[4] = np.ones((2, 2), dtype=int)

def movement_possible(tunnel, coords, rock_type):
    # For each of the cells occupied by the rock, check if the tunnel is empty
    for row in range(rocks[rock_type].shape[0]):
        for col in range(rocks[rock_type].shape[1]):
            if rocks[rock_type][row, col] == 1 and tunnel[coords[0]+row, coords[1]+col] == 1:
                return False
    return True
        

for rock in range(num_rocks):
    rock_type = rock % 5
    highest_rock = np.argmax(np.sum(tunnel, axis=1)!=0)

    # If highest rock is 0, then the tunnel is empty
    if highest_rock == 0:
        highest_rock = 1
    
    # Top coordinate of the rock
    top = highest_rock - 3 - rocks[rock_type].shape[0]

    # If the rock is too high, add empty rows at the top
    if top < 0:
        tunnel = np.vstack((np.zeros((-top, tunnel.shape[1]), dtype=int), tunnel))
        highest_rock += -top
        coords = (0, 2)
    else:
        coords = (top, 2)

    while True:
        # Move left or right
        if data[move % len(data)] == '>' and coords[1] + rocks[rock_type].shape[1] < tunnel.shape[1] and movement_possible(tunnel, (coords[0], coords[1]+1), rock_type):
            coords = (coords[0], coords[1]+1)
        elif data[move % len(data)] == '<' and coords[1] > 0 and movement_possible(tunnel, (coords[0], coords[1]-1), rock_type):
            coords = (coords[0], coords[1]-1)
        move += 1
        
        # Move down
        if coords[0]+1 + rocks[rock_type].shape[0] <= tunnel.shape[0] and movement_possible(tunnel, (coords[0]+1, coords[1]), rock_type):
            coords = (coords[0]+1, coords[1])
        else:
            break
    # With the final coordinates, add the rock to the tunnel
    tunnel[coords[0]:coords[0]+rocks[rock_type].shape[0], coords[1]:coords[1]+rocks[rock_type].shape[1]] += rocks[rock_type]
    
height = tunnel.shape[0] - np.argmax(np.sum(tunnel, axis=1)!=0)
print(f'After {num_rocks} the tower is {height} units high.')

After 2022 the tower is 3161 units high.


## Puzzle 2

In [3]:
import numpy as np

tunnel = np.zeros((1, 7), dtype=int)

num_rocks = 20000

move = 0
d_h = []
max_cycle = 1
cycle = (0,0)

rocks = {}
# rock_movements = {i: [] for i in range(5)}

rocks[0] = np.ones((1, 4), dtype=int)
rocks[1] = np.array([[0,1,0], [1,1,1], [0,1,0]], dtype=int)
rocks[2] = np.array([[0,0,1], [0,0,1], [1,1,1]], dtype=int)
rocks[3] = np.ones((4, 1), dtype=int)
rocks[4] = np.ones((2, 2), dtype=int)

def movement_possible(tunnel, coords, rock_type):
    # For each of the cells occupied by the rock, check if the tunnel is empty
    for row in range(rocks[rock_type].shape[0]):
        for col in range(rocks[rock_type].shape[1]):
            if rocks[rock_type][row, col] == 1 and tunnel[coords[0]+row, coords[1]+col] == 1:
                return False
    return True


old_height = 0
  

for rock in range(num_rocks):
    rock_type = rock % 5
    highest_rock = np.argmax(np.sum(tunnel, axis=1)!=0)

    # If highest rock is 0, then the tunnel is empty
    if highest_rock == 0:
        highest_rock = 1
    
    # Top coordinate of the rock
    top = highest_rock - 3 - rocks[rock_type].shape[0]

    # If the rock is too high, add empty rows at the top
    if top < 0:
        tunnel = np.vstack((np.zeros((-top, tunnel.shape[1]), dtype=int), tunnel))
        highest_rock += -top
        coords = (0, 2)
    else:
        coords = (top, 2)

    # rock_trajectory = ''
    while True:
        # Move left or right
        if data[move % len(data)] == '>' and coords[1] + rocks[rock_type].shape[1] < tunnel.shape[1] and movement_possible(tunnel, (coords[0], coords[1]+1), rock_type):
            coords = (coords[0], coords[1]+1)
        elif data[move % len(data)] == '<' and coords[1] > 0 and movement_possible(tunnel, (coords[0], coords[1]-1), rock_type):
            coords = (coords[0], coords[1]-1)
        move += 1
        
        # Move down
        if coords[0]+1 + rocks[rock_type].shape[0] <= tunnel.shape[0] and movement_possible(tunnel, (coords[0]+1, coords[1]), rock_type):
            coords = (coords[0]+1, coords[1])
        else:
            break
    # With the final coordinates, add the rock to the tunnel
    tunnel[coords[0]:coords[0]+rocks[rock_type].shape[0], coords[1]:coords[1]+rocks[rock_type].shape[1]] += rocks[rock_type]

    new_height = tunnel.shape[0] - np.argmax(np.sum(tunnel, axis=1)!=0)
    delta_h = new_height - old_height
    d_h.append(delta_h)

    old_height = new_height
    if move > 2*len(data):
        for i in range(max_cycle, rock // 3):
            if d_h[rock-i] == d_h[rock] and d_h[rock-i+1:rock] == d_h[rock-2*i+1:rock-i] and d_h[rock-i*2] == d_h[rock] and d_h[rock-i+1:rock] == d_h[rock-i*3+1:rock-i*2]:
                if i > max_cycle:
                    max_cycle = i
                    cycle = (rock-i*3+1, rock-i*2)
                    print(f'found 3 times repetition between {rock-3*i+1}-{rock-2*i}, {rock-2*i+1}-{rock-i}, {rock-i+1}-{rock}, a cycle adds {sum(d_h[rock-i+1:rock])} to the height.')

    
height = tunnel.shape[0] - np.argmax(np.sum(tunnel, axis=1)!=0)
print(f'After {num_rocks} the tower is {height} units high.')
print(f'The maximum cycle is {max_cycle} between rocks {cycle[0]} and {cycle[1]}.')

found 3 times repetition between 3893-3894, 3895-3896, 3897-3898, a cycle adds 1 to the height.
found 3 times repetition between 282-2026, 2027-3771, 3772-5516, a cycle adds 2747 to the height.
found 3 times repetition between 282-3771, 3772-7261, 7262-10751, a cycle adds 5497 to the height.
found 3 times repetition between 282-5516, 5517-10751, 10752-15986, a cycle adds 8247 to the height.
After 20000 the tower is 31505 units high.
The maximum cycle is 5235 between rocks 282 and 5516.


In [4]:
num_rocks = 1000000000000

start_cycle = cycle[0]
end_cycle = cycle[1]
init_height = sum(d_h[:start_cycle])

# Metric per cycle
cycle_height = sum(d_h[start_cycle:end_cycle+1])
cycle_delta = end_cycle - start_cycle +1

# Number of cycles
num_cycles = (num_rocks-start_cycle) // (cycle_delta)
remaining_rocks = num_rocks - start_cycle - num_cycles*(cycle_delta)
print(f'After {num_cycles} cycles (each containing {cycle_delta} rocks and adding {cycle_height} to the height) there are still {remaining_rocks} rocks.')

After 191021967 cycles (each containing 5235 rocks and adding 8250 to the height) there are still 2473 rocks.


In [5]:
final_height = init_height + num_cycles*cycle_height + sum(d_h[start_cycle:start_cycle+remaining_rocks])
print(f'Height before cycles: {init_height}')
print(f'Height added by all cycles: {num_cycles*cycle_height}')
print(f'Height added by extra rocks: {sum(d_h[start_cycle:start_cycle+remaining_rocks])}')
print(f'The final height is {final_height}.')

Height before cycles: 422
Height added by all cycles: 1575931227750
Height added by extra rocks: 3904
The final height is 1575931232076.
